# Vectorized Multivariate Linear & Polynomial Regression

This week's notebook will be heavy on theory, with a lot of different mathematics coming together to solve a seemingly simple problem. It will however also introduce several of the most important concepts in all of machine learning; modelling noise in the data, model complexity, and the resulting bias-variance trade-off. In addition, we'll use the tools from linear algebra to write much more compact and efficient versions of the linear model. Expect this programming assignment to take longer than previous notebooks to figure out, but afterwards you should also have a much better perspective on what machine learning is about.

Let's dive right in. The seemingly simple problem for this week is another regression data set, consisting of 30 data points. As you've now had an introduction to matrices, we'll call this data set what it is; a $30 \times 2$ data matrix. Below is the code to load the data.


In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = np.loadtxt('nonlinear_data.csv', delimiter=',', dtype=float)

# Print the data matrix
print(data.shape)
print(data)

## Nonlinear data

This exactly the same type of data set as we used last week, with one important exception: The relationship between $x$ and $y$ is *probably not linear*. The language here is a little careful, because it can be hard to just see just by looking at the data what the right kind of model is. Let's plot the data, so you can take a better look and make up your own mind.

The code below first splits the data into a vector for $x$ and a vector for $y$. The notation `[:, 0, None]` selects the first column from the data, while keeping it a *column* vector. The version `[:, 0]` would do the same, but automatically convert it to a *row* vector (i.e. a single array). Having $x$ and $y$ as *column* vectors will be useful for later steps, so we'll just split them like this here already.

In [ ]:
# Slice out the column vectors x and y
x = data[:, 0, None]
y = data[:, 1, None]

# Plot the data
plt.figure(figsize=(12,9))
plt.plot(x, y, 'r.')
plt.show()

### Assignment 1: Adding $x_0$ to the data

The first step of making a vectorized version of regression, is to add an extra feature $x_0$ to the data. This feature should just be $1$ for every data point and will simplify the computations in later steps.

Write function `add_x0`, which takes a column vector `x` and adds a column vector of ones in front of it. The function should return the combined matrix, with the first column being the new feature $x_0$ and the second column being the original $x_1$ feature.

For this function you should use [np.ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) to create the new feature and [np.hstack](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html) to combine the new and old feature vectors. Inspect the resulting matrix and ensure it looks correct before moving on to the next step.


In [ ]:
def add_x0(x):
    # YOUR CODE HERE
    

X = add_x0(x)
print(X)

assert X.shape == (30, 2) and X[0,0] == 1, 'X matrix seems incorrect'

### Assignment 2: Linear model

Even if a model does not look linear, a simple linear model is still a good place to start. For now, let's assume we already know what our $\theta$ parameter is, and are just trying to predict the hypothesis value for every value of $x$. In the previous module's notebook we wrote a simple function and a loop to apply the linear model to every x-value. However, we could also view the computing the hypothesis values for the entire data set as a **system of linear equations**:

$$x^1_0 \theta_0 + x^1_1 \theta_1 = h_{\theta}(x^1)$$
$$x^2_0 \theta_0 + x^2_1 \theta_1 = h_{\theta}(x^2)$$
$$x^3_0 \theta_0 + x^3_1 \theta_1 = h_{\theta}(x^3)$$ 
$$\dots$$
$$x^m_0 \theta_0 + x^m_1 \theta_1 = h_{\theta}(x^m)$$

This exactly the type of system of equations we can model using *linear algebra* as a single matrix multiplication:

$$ \left[\begin{array}{cccc}
x^1_0 & x^1_1 \\ 
x^2_0 & x^2_1 \\
x^3_0 & x^3_1 \\ 
\vdots & \vdots \\
x^m_0 & x^m_1 \\ 
\end{array} \right]
\left[\begin{array}{c} \theta_0 \\ \theta_1 \end{array} \right]
= \left[\begin{array}{c} h_{\theta}(x^1) \\ h_{\theta}(x^2) \\
h_{\theta}(x^3) \\ \vdots \\ h_{\theta}(x^m) \end{array} \right]$$

The equations above and the matrix multiplication are completely equivalent. The matrix notation is just a more compact (and faster) way to compute exactly the same $m$ hypothesis values for some value of $\theta$ in a single operation.

The parameter vector $\theta$ should be a column vector, and for this linear model it should just have dimensions $2 \times 1$. Make a $\theta$ vector with starting values of 1 for every parameter, so:

$$\theta = \left[\begin{array}{c} 1 \\ 1 \end{array} \right]$$

You can use [np.array](https://numpy.org/doc/stable/reference/generated/numpy.array.html) to create $\theta$ or use the `np.ones` function from before. Write the function `linear_model` which takes a matrix `X` and vector `theta` and returns the resulting column vector of hypothesis values. Use the function [np.dot](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) for the matrix multiplication operation.


In [ ]:
def linear_model(X, theta):
    # YOUR CODE HERE
    

# Create the starting vector for theta
# theta = ...
# YOUR CODE HERE

# Compute the linear model
h = linear_model(X, theta)

# Plot the model results
plt.figure(figsize=(12,9))
plt.plot(x, y, 'r.')
plt.plot(x, h)
plt.show()

## Multivariate linear regression

It might seem as if we are building the exact same model as last week, because we are. There is one key difference though; *generality*. The model you just wrote, actually would work just as well for multivariate linear regression. Only the dimensions of $\theta$ and $X$ would change, but the actual code would not.

The system of equations for a multivariate linear regression with $n$ features, would look like:

$$x^1_0 \theta_0 + x^1_1 \theta_1 + \dots + x^1_n \theta_n = h_{\theta}(x^1)$$
$$x^2_0 \theta_0 + x^2_1 \theta_1 + \dots + x^2_n \theta_n = h_{\theta}(x^2)$$
$$x^2_0 \theta_0 + x^2_1 \theta_1 + \dots + x^2_n \theta_n = h_{\theta}(x^2)$$
$$\dots$$
$$x^m_0 \theta_0 + x^m_1 \theta_1 + \dots + x^m_n \theta_n = h_{\theta}(x^m)$$

And the corresponding matrix multiplication:

$$ \left[\begin{array}{cccc}
x_0^1 & x_1^1 & x_2^1 & \cdots & x_n^1 \\ 
x_0^2 & x_1^2 & x_2^2 & \cdots & x_n^2 \\ 
x_0^3 & x_1^3 & x_2^3 & \cdots & x_n^3 \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
x_0^m & x_1^m & x_2^m & \cdots & x_n^m \\ 
\end{array} \right]
\left[\begin{array}{c} \theta_0 \\ \theta_1 \\
\theta_2 \\ \vdots \\ \theta_n \end{array} \right]
= \left[\begin{array}{c} h_{\theta}(x^1) \\ h_{\theta}(x^2) \\
h_{\theta}(x^3) \\ \vdots \\ h_{\theta}(x^m) \end{array} \right]$$

In the compact language of linear algebra, both the univariate and multivariate linear models can just be written as:

$$X \theta = h_{\theta}(X)$$

The only thing that changes between these models is the exact dimensions of $X$ and $\theta$, depending on the number of features used.

For now we'll just continue with univariate linear regression here, because it is really hard to plot anything in $n$ dimensions. It is good to keep in mind though that if you write all your functions to work with vectors and matrices, _changing the number of features in the model will actually be trivial_.

### Assignment 3: Cost function

The next part of the linear model will be writing the cost function:

$$ J(\theta) = \frac{1}{2m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)^2$$

Write the entire function `linear_cost` using just matrix and vector operations, so *no loops*! Use your `linear_model` function to compute the vector $h_{\theta}(X)$. You may use [np.square](https://numpy.org/doc/stable/reference/generated/numpy.square.html) and [np.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) to write the function, but with some careful thought, it is actually possible to write the whole thing with just pure linear algebra operations, which is good practice.

In [ ]:
def linear_cost(theta, X, y):
    # YOUR CODE HERE
    

assert np.isclose(linear_cost(theta, X, y), 263.709614), 'Result of cost function seems incorrect'

### Assignment 4: Gradient Vector

The partial derivatives for each of the $\theta$ parameters, using the exact same derivative rules as last week. The only difference being that there are now $n+1$ partial derivatives for each of the $n+1$ parameters. The resulting set of equations looks like this:
 
$$\frac{\partial J}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_0$$
$$\frac{\partial J}{\partial \theta_1} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_1$$
$$\frac{\partial J}{\partial \theta_2} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_2$$
$$\dots$$
$$\frac{\partial J}{\partial \theta_n} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_n$$

Make sure you understand how this relates to the two partial derivative equations from last week, as they should seem very similar. All these partial derivatives can be combined into a single gradient vector, like so:

$$\frac{\partial J}{\partial \theta} = \left[\begin{array}{c} \frac{\partial J}{\partial \theta_0} \\ \frac{\partial J}{\partial \theta_1} \\
\frac{\partial J}{\partial \theta_2} \\ \vdots \\ \frac{\partial J}{\partial \theta_n} \end{array} \right]$$

This is really just an $(n+1) \times 1$ column vector of all the partial derivative. In fact, we can again use the tools of linear algebra to compute the whole vector as a couple of operations, without using any loops. Write the function `gradient_vector` to construct this vector without loops.

**Hint:** Writing the gradient vector as a matrix operation can be tricky. Any operation where you are doing multiplications with elements and summing them afterwards should make you think of trying to do this with a matrix multiplication. It can be a lot easier to think about matrix multiplication in terms of the dimensions of the matrices going in and coming out, rather than to think of the specific values of each matrix element. Here the result should be a column vector of partial derivatives, so dimensions $(n+1) \times 1$, with $n+1$ being the number of parameters of your model. In case of simple univariate linear regression, this is just a $2 \times 1$ vector.

In [ ]:
def gradient_vector(theta, X, y):
    # YOUR CODE HERE
    

assert np.isclose(gradient_vector(theta, X, y), np.array([[-7.587254],
    [-8.349297]])).all(), 'Result of gradient function seems incorrect'

### Assignment 5: Gradient Descent

With the gradient vector code done, we can start with the actual `gradient_descent` function. You can (and should) take inspiration from your gradient descent code from the previous module's notebook. Include checks for when the cost is diverging and when the algorithm has converged. The only real difference is that $\theta$ parameter is now a vector, instead of separate variables for $a$ and $b$. With the gradient already returned as column vector too, doing the gradient descent update for all the parameters can actually neatly be be done all at once as a single operation.
 

In [ ]:
def gradient_descent(X, y, theta, alpha, thres=10**-6):
    # YOUR CODE HERE
    

# Find the theta vector that minimizes the cost function
theta_hat = gradient_descent(X, y, theta, 0.9)
print("\nFound the following values for theta:")
print(theta_hat)

# Plot the model results
print("\nResulting in the following model:")
h = linear_model(X, theta_hat)
plt.figure(figsize=(12,9))
plt.plot(x, y, 'r.')
plt.plot(x, h)
plt.show()

## Noise

Intuitively, the model above probably doesn't look like a good fit, even it *is* the best possible linear line according to the cost function. In this next part of the assignment, we're going to change gears a bit and try to get a better perspective on how *good* or *bad* the fit of a model is exactly. For this we'll extend the model to include the *noise* within the data, so we can be more specific about how much we expect the data to differ from our fitted line model.

Noise is _any (random) variation_ in the data that is not a part of the underlying pattern you are trying to model. There are many possible causes for noise, the most straightforward option is that whatever instrument or measurement that was used to collect the data is not perfectly accurate. Random variations in the data can just be a result of imprecise measurement, but more commonly there are actually just other factors that should be part of the data weren't, or even *can't* be directly measured. If you don't have data for a factor that *does* actually affect the target variable, then your model can never capture the relationship between this factor and the target variable, therefore the effect of this factor should be considered noise.

Let's say you're trying to predict housing prices in Amsterdam, and you start off with a regression model based just on the number of square meters. You can probably make a decent prediction for the price of a house using this model, but more features would make your model better. For instance, you could include distance to Dam square from the house as a second feature, and the predictions would probably become quite a bit more accurate. You keep adding more and more features about the exact details of each house, but then you realise the state of the economy and the housing market at the time of sale will probably also be a relevant factor. Now could also start adding in all sorts of stock market indices like *AEX* points, at each time of sale, to try and capture this aspect. This is going to make your data harder to collect, and probably it will *still* be incomplete after this. This is the most common source of noise; factors that affect the target variable that you can't or won't directly measure. 

## Gaussian noise distribution

The target variable in regression is a continuous value, so any random noise added to the data should also come from a continuous distribution. By far the most common continuous probability distribution used to model data is the *Gaussian* distribution, also called the *Normal* distribution. It turns out many things in nature can be approximated with Gaussians and it is a relatively simple model to use.

For this model we'll assume that there is an underlying function $g$ that pefectly captures the relationship between the variables $x$ and $y$ using some function. However, after the model is computed, some random noise drawn from a Gaussian distribution is added to the function output, meaning the measured values for some particular $x^i$ and $y^i$ will not perfectly fit the pattern of $g$. The Gaussian distribution for this noise has a $0$ mean and a variance of $\sigma^2$:

$$y^i \sim g(x^i) + \mathcal{N}(0, \sigma^2)$$

This means the noise is symmetrical around the *true* underlying function $g$ and the only parameter defining the amount of noise is $\sigma$, which indicates how "wide" the spread of noise is.

*One of the fundamental goals in machine learning is to try and learn a function $h_{\theta}$ that matches the true function $g$ as closely as possible, without including the noise.*

### Assignment 6: Computing residuals and estimating sigma

Now, we can simply assume that the hypothesis function $h_{\theta}$ does indeed match $g$, because it minimizes the cost function. If we replace $g$ with $h_{\theta}$ and move it to the left hand side, we get:

$$y^i - h_{\theta}(x^i) \sim \mathcal{N}(0, \sigma^2)$$

The left hand side is now what is called the residual of the model; whatever is still left that the model didn't capture:

$$r^i = y^i - h_{\theta}(x^i)$$

These residuals are _exactly_ the noise component of the data, and so are assumed to have a 0 mean. Estimating the standard deviation for the residual vector then becomes:

$$\sigma = \sqrt{\frac{1}{m-1} \sum_{i=1}^m (r^i - 0)^2}$$

Write a function `compute_residual_std_dev` to compute the vector of residuals, based on a `y` vector of target values and an `h` vector of hypothesis values. The function should return the standard deviation of this residuals vector, assuming a 0 mean. You should be able to borrow some code from the `linear_cost` function for this, as the computations are really quite similar... Use [np.sqrt](https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html) for the square root computation.

In [ ]:
from scipy.stats import norm

def compute_residuals_std_dev(y, h):
    # YOUR CODE HERE


assert np.isclose(compute_residuals_std_dev(y, h), 15.653720), 'Residuals std dev seems incorrect'

# Compute values required for the plot
sigma = compute_residuals_std_dev(y, h)
samps = np.linspace(-3.5*sigma, 3.5*sigma, 200)
probs = norm.pdf(samps, scale=sigma)

# Draw the Gaussian distribution
plt.rc('axes', labelsize=24)
plt.figure(figsize=(12,9))
plt.plot(samps, probs)
plt.plot(y - h, np.zeros(y.shape), 'r.', alpha=0.3)

# Draw sigma bounds within the distribution
plt.plot([0,0], [0,norm.pdf(0, scale=sigma)], 'g')
plt.plot([-sigma,-sigma], [0,norm.pdf(sigma, scale=sigma)], 'y')
plt.plot([sigma,sigma], [0,norm.pdf(sigma, scale=sigma)], 'y')
plt.plot([-3*sigma,-3*sigma], [0,norm.pdf(3*sigma, scale=sigma)], 'm')
plt.plot([3*sigma,3*sigma], [0,norm.pdf(3*sigma, scale=sigma)], 'm')

# Label things nicely
plt.title('Gaussian model of the residuals', fontsize=32)
plt.xlabel('Y-axis spread of points around the fitted line')
plt.ylabel('Probability density')
plt.show()

## Probability density model for the noise

The model plotted above indicates the spread of the individual data points on the y-axis, as compared to the fitted line, i.e. the residuals. The probability density function indicates how likely a value is to occur; the higher the probability density, the more likely the value. The mean residual value of 0 is therefore "most likely". This statement is not technically correct though, because the probability of any specific continuous value is always 0. We can never say anything about the probability of a specific value in continuous distribution, but we can be exact about *ranges* of values. For instance:

* With a probability of 0.6827 a point will have residual between $-\sigma$ and $\sigma$, indicated by the two yellow lines
* With a probability of 0.9973 a point will have residual between $-3\sigma$ and $3\sigma$, indicated by the two magenta lines

### 3D version of the model

Below is the code to create a slightly different version of this same probability density plot, but using 3 dimensions. No need to write any code for this, but I think the plot helps to better understand the model for the residuals.

The $x$ and $y$ axis for this plot correspond to the exact same $x$ and $y$ axis in the earlier fitted line plots. So the red dots are data points for $x$ and $y$ and the green line is the model prediction. For this part of the plot there is no $z$ dimension, and everything is projected flat at $z=0$.

The blue wire mesh does have a $z$ component, namely the probability density at a point $x,y$. This Gaussian distribution models the noise on the $y$ axis, around the fitted line. The mean of this Gaussian shifts with this fitted line, ensuring the noise is always centered around the line.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Create the 3D plot
ax = Axes3D(plt.figure(figsize=(16,12)))
xs = np.linspace(0, 1, 150)
ys = np.linspace(-75, 100, 150)

# Compute the probability density surface
xr, yr = np.meshgrid(xs, ys)
pdf_map = np.vectorize(lambda x,y,t,s: norm.pdf((t[0,0] + x*t[1,0]) - y, scale=s), excluded=(2, 3))
zr = pdf_map(np.ravel(xr), np.ravel(yr), theta, sigma).reshape(xr.shape)

# Plot the points and lines
ax.plot_wireframe(xr, yr, zr)
ax.scatter(x, y, c='r')
ax.plot(x, h, c='g')

# Label things
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('Probability density')
plt.show()

### Assignment 7: Plotting model certainty

For the next step, we're going to combine this theory and the previous two plots into a single 2D plot that should give a much clearer picture of the certainty of the learned model. Write the function `plot_model_certainty`, which takes 5 arguments:

* `x`, the vector of $x$ values of the data set.
* `y`, the vector of $y$ values of the data set .
* `std_dev`, the estimate for the standard deviation of the residuals comparing $y^i$ and $h_(x^i)$.
* `x_samples`, a vector of evenly spaced samples on the x-axis, used to show the model predictions at every point.
* `y_predicts`, a vector of computed predictions, corresponding each of the `x_samples` values.

Write the function `plot_model_certainty` to create a plot with the following attributes:

* The original data point should be red dots in the plot.
* The predicted line for the learned model, in green.
* A yellow bounding line $1\sigma$ *above* the predicted line, following the same shape as the predicted line.
* A yellow bounding line $1\sigma$ *below* the predicted line, following the same shape as the predicted line.
* A magenta bounding line $3\sigma$ *above* the predicted line, following the same shape as the predicted line.
* A magenta bounding line $3\sigma$ *below* the predicted line, following the same shape as the predicted line.

The resulting plot should give a clear picture of how certain the model is that each of the data points fall in a certain range, adding *confidence bounds* to the predicted line. The probabilities for points falling between the yellow and magenta lines are the same as in the first Gaussian plot.


In [ ]:
def plot_model_certainty(x, y, std_dev, x_samples, y_predicts):
    plt.figure(figsize=(12,9))
    
    # YOUR CODE HERE
    

# Create a column vector of 100 evenly spaced x-axis samples
samps = np.expand_dims(np.linspace(0, 1, 100), axis=1)

# Compute the model predictions for all 100 samples
preds = linear_model(add_x0(samps), theta_hat)

# Plot model certainty with confidence bounds
plot_model_certainty(x, y, sigma, samps, preds)

#### Sidenote: Maximum Likelihood Estimation

This Gaussian modelling is not just useful for adding confidence bounds to a model, but also has a more fundamental function too, which it seems a waste to not at least mention. We won't go into the details of showing it here, but exactly this Gaussian model for noise, can also be used try and find the parameter vector $\theta$ that is *most likely* given the data. Formally, this called the *Maximum Likelihood Estimator*. It turns out, that finding the MLE according to this model of the noise, is actualy **identical** to finding the $\theta$ vector that minimizes the cost function. This bit of theory links the probabilistic prespective of the data together with the minimizing the cost function, and gives a good theoretical reason as to why this particular cost function is a good one to use.

### Assignment 8: Polynomial feature expansion

Now that we have a multivariate linear regression model and way to show the confidence bounds of the model, we can get started with actually improving those confidence bounds. The multivariate linear model can be used to include other input features, but alternatively, we can create some new *nonlinear features* from the existing feature and add those the model inputs. Somewhat confusingly, the model itself is then still just multivariate *linear* regression, as we've only really changed what the input features for that linear model are.

The most common feature expansion used in regression is the polynomial feature expansion, where the input is expanded with a squared, cubic, quadratic, etc. feature, in order to learn polynomials of a higher degree. The original univariate $x$ vector was:

$$ x = \left[\begin{array}{c}
x^1\\ 
x^2\\
x^3\\ 
\vdots\\
x^m\\ 
\end{array} \right]$$

Which we can then expand to a multivariate input matrix with $n^{th}$ degree features like so:

$$ X = \left[\begin{array}{cccc}
1 & x^1 & (x^1)^2 & \cdots & (x^1)^n \\ 
1 & x^2 & (x^2)^2 & \cdots & (x^2)^n \\
1 & x^3 & (x^3)^2 & \cdots & (x^3)^n \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x^m & (x^m)^2 & \cdots & (x^m)^n \\ 
\end{array} \right]$$

The notation here is a bit confusing, because of how the indexing is done. Some examples for clarity:

* $(x^1)^2$ is the square of the original first x-value $x^1$, i.e. $x^1 \cdot x^1$.
* $(x^m)^3$ would be the cube of the original last x-value $x^m$, i.e. $x^m \cdot x^m \cdot x^m$

Write the function `polynomial_features`, which takes a feature vector `x` and returns a $m \times (n+1)$ matrix of polynomial features, which the highest degree being `n`. Reuse the function `add_x0` from before to get the matrix started and incrementally add higher degree vectors using [np.hstack](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html). Each of the higher degree vectors can be computed using [np.power](https://numpy.org/doc/stable/reference/generated/numpy.power.html).

In [ ]:
def polynomial_features(x, n):
    # YOUR CODE HERE
    

X_third_degree = polynomial_features(x, 3)
print(X_third_degree)

### Assignment 9: Putting it all together

We now actually already have all the components to fit these higher degree polynomials, as we can just reuse all of the multivariate linear code from before. Write the function `plot_poly_descent`, which takes a vector `x`, a vector `y`, a maximum `degree` for the model, and learning rate `alpha`, and does the following:

1. Create the polynomial feature matrix of `x` to the correct `degree`.
2. Create a starting vector for $\theta$, of the correct dimensions, containing all *ones*.
3. Apply gradient descent to the polynomial feature matrix, using the starting vector $\theta$ and the learning rate `alpha`, and store the resulting best $\theta$ vector.
4. Compute the vector $h_{\theta}(X)$ with the new learned $\theta$.
5. Compute the standard deviation of the residuals of this model.
6. Create a column vector of 100 samples evenly spaced between 0 and 1.
7. Compute the model prediction for these samples using the same polynomial feature expansion and learned $\theta$ vector.
8. Create a plot of the model, include the confidence bounds, using all these elements.


In [ ]:
def plot_poly_descent(x, y, degree, alpha):
    # YOUR CODE HERE
    


print("\nFitted line and confidence bounds for degree 3 polynomial")
plot_poly_descent(x, y, 3, 0.9)

print("\nFitted line and confidence bounds for degree 5 polynomial")
plot_poly_descent(x, y, 5, 0.9)

## The Normal equation

If everything worked correctly, you should see plots for the fitted lines using $3^{rd}$ and $5^{th}$ degree polynomials. You should now be able to create polynomials of *any* degree using this code and find the best fitting $\theta$ for that degree. The downside of the gradient descent approach though, is that the learning rate probably needs to set separately for each of the higher degree polynomials. Additionally, with some of the lower learning rates, it might take a very long time for the algorithm to converge.

Specifically for multivariate linear regression, there is an alternative solution for learning $\theta$. For most models, gradient descent is the only option, but here we have another solution, called the *Normal equation*:

$$\theta = (X^T X)^{-1}X^Ty$$

This is a set of linear algebra operations that finds the value for $\theta$ which solves the following system of equations:

$$\left[\begin{array}{c} \frac{\partial J}{\partial \theta_0} \\ \frac{\partial J}{\partial \theta_1} \\
\frac{\partial J}{\partial \theta_2} \\ \vdots \\ \frac{\partial J}{\partial \theta_n} \end{array} \right]= \left[\begin{array}{c} 0\\ 0\\ 0 \\ \vdots \\ 0 \end{array} \right]$$

So the Normal equation solves for what value of $\theta$ the entire gradient vector is equaly to zero, i.e. the minimum of the cost function. How exactly this system of equations is solved by the Normal equation is really beyond the scope of this course, but it will still be a useful tool here. The function `normal_fit` implements the Normal equation and has already been given. It returns the best fitting parameter $\theta$ for a matrix `X` and a vector `y`.

### Assignment 10: Plotting the Normal fit

Write the function `plot_poly_fit`, which should do exactly the same thing as `plot_poly_descent`, but replace the gradient descent portion with `normal_fit`. This way the code does not depend on a specific learning rate, and finding the best vector $\theta$ should be much faster this way.

In [ ]:
def normal_fit(X, y):
    return np.dot(np.dot(np.linalg.pinv(np.dot(X.T, X)), X.T), y)

def plot_poly_fit(x, y, degree):
    # YOUR CODE HERE


print("\nFitted line and confidence bounds for degree 10 polynomial")
plot_poly_fit(x, y, 10)

print("\nFitted line and confidence bounds for degree 30 polynomial")
plot_poly_fit(x, y, 30)

## Validation data

The last portion of this assignment we will focus on answering the question:

* Which degree of polynomial fits this data the best?

Increasing the degree of the polynomial will always add more parameters to the model, as the dimensions of $\theta$ also increase. More parameters means the model is more "flexible", as there are more values to change and create the *perfect fit*. This means, as you increase the the degree of the model, the cost on the training data will always decrease, and the confidence bounds will always move closer together.

So is the perfect model just when the line moves exactly through all the data points and the confidence bounds are completely on top of the fitted line? No, because of the noise in the data, perfectly fitting all the data points will probably not generalize very well to new points. You can actually see a clear example of this in the degree 30 polynomial plot above. Between the last and second to last points, the function makes a strange dip, that doesn't correspond to any data point. This the best way for the polynomial to exactly fit those 2 points apparently, but it doesn't capture any part of the underlying trend *between* the two points.

This type of model is said to be overfitting. It is fitting the noise in the data and no longer approximating the true function $g$. _You can detect overfitting by testing with data points you didn't train on._

### Assignment 11: K-fold cross-validation

In the previous assignments we used a separate test set to better estimate the performance of a learned model on new data points. A validation set is very similar, but serves a slightly different purpose. It is also a portion of the data that *isn't* used when training, but it is instead used to select the best of several trained models. If you then, after selecting the best model using the validation set, want an objective assessment of the performance of this model on new data, you would use a testing set. In proper machine learning research, you would always use all 3; training set, validation set, and testing set. For this assignment we'll skip the testing portion and only do validation to select the best degree polynomial model.

Splitting the data even further into a training and validation set, especially for a data set this small, has some challenges. The bigger you make your training set, the more data to fit your model parameters, so the better the learned model will be. However, the bigger you make your validation set, the better the estimate of the performance of the learned model. A classic solution to this trade-off is doing what is called *k-fold cross-validation*. You shuffle the data and divide it into $k$ equal parts (or folds). Then you repeat the training and validation process $k$ times, each time using a different part as the validation set, and all the other $k-1$ parts as the training set.

A common option is using $k=5$, which means you get 5 different train and validation splits, each consisting of $80\%$ training and $20\%$ validation. You can then average the training and testing costs for each of the 5 iterations, giving a better overal estimate. As an added benefit, during this whole process, every data point also gets used as a validation point exactly once, again improving the reliability of the estimates.

The function `k_fold_validation` below has already been mostly completed. Take a look at the code provided to see if you can understand how the different folds are constructed. Complete the last portion of the function to actually compute the average training and validation costs for all the folds and return these averages.

In [ ]:
def k_fold_validation(X, y, k=5):
    # Stack X and y together, so they get shuffled in the same way
    D = np.hstack((X, y))
    # Shuffle the rows of the combined data matrix D
    np.random.shuffle(D)
    
    # Compute the size of each of the folds
    fold_size = int(round(D.shape[0] / k))
    
    avg_train_cost = 0 
    avg_val_cost = 0
    # For each of the K folds
    for i in range(k):
        # Slice out the i^th validation section of fold_size number of rows
        x_val = D[i*fold_size:(i+1)*fold_size, :-1]
        # Slice out the corresponding validation y (column) vector
        y_val = D[i*fold_size:(i+1)*fold_size, -1, None]
        
        # Slice out the sections before and after the validation section
        # Stack these back together to form the training data
        x_train = np.vstack((D[:i*fold_size, :-1], D[(i+1)*fold_size:, :-1]))
        # Do the same for the y training vector
        y_train = np.vstack((D[:i*fold_size, -1, None], D[(i+1)*fold_size:, -1, None]))
        
        # Find the best fitting theta for this training set using the Normal equation
        theta = normal_fit(x_train, y_train)

        # YOUR CODE HERE
        

    return avg_train_cost, avg_val_cost

training_cost, validation_cost = k_fold_validation(X_third_degree, y, 5)
print("Average training cost:", training_cost)
print("Average validation cost:", validation_cost)

### Assignment 12: Model selection

Usually k-fold cross validation is enough to get a good estimate of the training and validation costs for a model. However, in this particular case the data set is *very* small, and for the next plot to make sense, we'll need really reliable estimates. The best way to do that is just repeat k-fold cross validation a number of times, with the the data being shuffled at random before every split into folds, and average those results again.

Write the function `compute_train_val_costs`, which estimates the training and validation costs for a range of different polynomial degree models between 1 and `max_degree`. The function should return 3 lists, one simple list of the used degrees, and two other lists of the same length; one containing the average training cost for each degree and one containing the average validation cost for each degree.

Your function should construct the appropriate polynomial feature matrix for each degree and then repeat the 5-fold cross-validation process `iters` number of times. Each resulting training and validation cost should be averaged to a single training and a single validation cost for every polynomial degree tried.

In [ ]:
def compute_train_val_costs(x, y, max_degree, iters):
    # Create list of degrees and two cost lists
    degrees = list(range(1, max_degree+1))
    train_costs =  []
    validation_costs = []
    
    for deg in degrees:
        # YOUR CODE HERE
        
    
    return degrees, train_costs, validation_costs


degrees, train_costs, val_costs = compute_train_val_costs(x, y, 10, 1000)

# Plot with all the results
plt.figure(figsize=(12,9))
plt.plot(degrees, train_costs, label='training cost')
plt.plot(degrees, val_costs, label='validation cost')
plt.title('Underfitting vs. Overfitting', fontsize=32)
plt.xlabel('Polynomial degree')
plt.ylabel('Cost')
plt.legend(fontsize=18)
plt.show()

## Underfitting and overfitting

Understanding the plot above is essential. The x-axis shows the degree of the polynomial model, which corresponds to the complexity of that model; the more parameters the model has the more complex the learned model can be. There are 3 distinct areas in this plot:

* Degree 1-3: Training cost is high and validation cost is high. This is called **underfitting**. Here the model does not have enough complexity to actually fit the underlying function $g$ (like with the original degree 1 model). This means the training cost can never get low and produce a good fit, because there is no good solution for a model that is too simple.
* Degree 7-10: Training cost is very low and validation cost is high. This is called **overfitting**. Here the model is too complex and too flexible, so it will start to actually fit the noise. With enough flexibility the model can fit *any* function, include the random pattern in the noise. This will create a very low training cost, but won't generalize well to other data points (like in the validation set), so the cost on the validation set becomes very high.
* Degree 4-6: Training cost is low and validation cost is low: A good fit. Degree 4 seems to fit best, having the lowest validation cost. Here the model is flexible enough to fit the true function $g$, but not so flexible as to also start fitting the noise. This is the ideal result for your model.

### Sidenote: Bias Variance trade-off

An underfit model is also called a *high bias* model and an overfit model is also called *high variance* model. These terms come from statistics and are often used instead of overfitting and underfitting, so it is good to know them. In fact, it is possible to formally define the bias and the variance of the model, and show that the cost for some set of data points only has 3 components:

$$\frac{1}{2m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)^2 = Bias^2 + Variance + \sigma^2$$

* The bias, corresponding to the amount of underfitting.
* The variance, corresponding to the amount of overfitting.
* $\sigma$, the standard deviation of the noise component.

The noise component of the cost cannot be reduced, as it is random, so actually *the only way to get a better model is to improve where you are in this underfitting-overfitting spectrum.*

### Assignment 13: Plotting the best fit

Use [np.argmin](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html) to find the minimum index of the validation costs list. Then use this index to find the corresponding degree for this minimum. Print the polynomial degree with the lowest validation cost. Use `plot_poly_fit` to plot the polynomial model with this best fitting degree.

In [ ]:
# YOUR CODE HERE
